In [3]:
import torch
from torch import nn

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cpu'

In [4]:
class FashionMNISTModelv1(nn.Module):
    def __init__(self, input_shape: int, hidden_units: int, output_shape:int):
        super().__init__()
        self.layer_stack = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=input_shape, out_features=hidden_units),
            nn.ReLU(),
            nn.Linear(in_features=hidden_units, out_features=output_shape),
            nn.ReLU()
        )

    def forward(self, x: torch.Tensor):
        return self.layer_stack(x)

In [6]:
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [7]:
train_data = datasets.FashionMNIST(root="data", train=True, download=True, transform=ToTensor(), target_transform=None)
test_data = datasets.FashionMNIST(root="data", train=False, download=True, transform=ToTensor())

BATCH_SIZE=32

train_dataloader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
test_dataloader = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

len(train_dataloader), len(test_dataloader)

(1875, 313)

In [9]:
class_names = train_data.classes
class_names

['T-shirt/top',
 'Trouser',
 'Pullover',
 'Dress',
 'Coat',
 'Sandal',
 'Shirt',
 'Sneaker',
 'Bag',
 'Ankle boot']

In [10]:
torch.manual_seed(42)

model_1 = FashionMNISTModelv1(input_shape=28*28, hidden_units=10, output_shape=len(class_names)).to(device)

In [11]:
from helper_functions import accuracy_fn

loss_fn = nn.CrossEntropyLoss()
optimizer=torch.optim.SGD(params=model_1.parameters(), lr=0.01)

In [26]:
def train_step(
    model: torch.nn.Module,
    data_loader: torch.utils.data.DataLoader,
    loss_fn: torch.nn.Module,
    optimizer: torch.optim.Optimizer,
    accuracy_fn,
    device: torch.device = device
    ):
    
    train_loss = 0.
    train_acc = 0.

    model.train()

    for batch, (X, y) in enumerate(data_loader):
        X, y = X.to(device), y.to(device)

        y_pred = model(X)
        loss = loss_fn(y_pred, y)
        train_loss += loss
        train_acc += accuracy_fn(y, y_pred.argmax(dim=1))
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    train_loss /= len(data_loader)
    train_acc /= len(data_loader)

    print(f"Train loss: {train_loss:.3f} | Train accuracy: {train_acc:.3f}")

In [22]:
def test_step(
    model: torch.nn.Module,
    data_loader: torch.utils.data.DataLoader,
    loss_fn: torch.nn.Module,
    accuracy_fn,
    device: torch.device = device
):

    test_loss = 0.
    test_acc = 0.

    model.eval()
    with torch.inference_mode():
        for batch, (X_test, y_test) in enumerate(data_loader):
            X_test, y_test = X_test.to(device), y_test.to(device)
            y_test_pred = model(X_test)
            test_loss += loss_fn(y_test_pred, y_test)
            test_acc += accuracy_fn(y_test, y_test_pred.argmax(dim=1))

        test_loss /= len(data_loader)
        test_acc /= len(data_loader)
    
        print(f"Test Loss: {test_loss:.3f} | Test accuracy: {test_acc:.3f}")

In [27]:
from tqdm.auto import tqdm
torch.manual_seed(42)
from timeit import default_timer as timer

start_time = timer()

n_epochs = 3
for epoch in tqdm(range(n_epochs)):
    print(f"Epoch: {epoch}")
    train_step(model_1, train_dataloader, loss_fn, optimizer, accuracy_fn)
    test_step(model_1, test_dataloader, loss_fn, accuracy_fn)

end_time = timer()
print(f"Total training time: {(end_time - start_time):.3f} seconds.")

  0%|          | 0/3 [00:00<?, ?it/s]

Epoch: 0
Train loss: 1.033 | Train accuracy: 64.487


 33%|███▎      | 1/3 [00:04<00:09,  4.52s/it]

Test Loss: 0.997 | Test accuracy: 65.056
Epoch: 1
Train loss: 0.960 | Train accuracy: 66.005


 67%|██████▋   | 2/3 [00:08<00:04,  4.47s/it]

Test Loss: 0.960 | Test accuracy: 65.485
Epoch: 2
Train loss: 0.929 | Train accuracy: 66.660


100%|██████████| 3/3 [00:13<00:00,  4.46s/it]

Test Loss: 0.943 | Test accuracy: 65.795
Total training time: 13.392 seconds.


In [30]:
def eval_model(
    model: torch.nn.Module,
    data_loader: torch.utils.data.DataLoader,
    loss_fn: torch.nn.Module,
    accuracy_fn,
    device: torch.device = device
):
    loss, acc = 0., 0.
    model.eval()

    with torch.inference_mode():
        for X, y in data_loader:
            X, y = X.to(device), y.to(device)
            y_pred = model(X)
            loss += loss_fn(y_pred, y)
            acc += accuracy_fn(y, y_pred.argmax(dim=1)) # logit to probabilies.

        loss /= len(data_loader)
        acc /= len(data_loader)

    return {
        "model": model.__class__.__name__,
        "loss": loss.item(),
        "accuracy": acc
    }

In [31]:
model_1_results = eval_model(model_1, test_dataloader, loss_fn, accuracy_fn)
model_1_results

{'model': 'FashionMNISTModelv1',
 'loss': 0.9431642889976501,
 'accuracy': 65.79472843450479}